This notebook is intended to test out the `mir_eval` functionality on heuristically-converted MIDI vs. ground truth MIDI.

In [ ]:
import pretty_midi
import os
import pickle
import numpy as np
import mir_eval
import matplotlib.pyplot as plt
from scipy.io.wavfile import write as wavwrite

The code below to the "debug" section is intended to be run consistently.

In [ ]:
hpath = '/juice/scr/rjcaste/curis/wavegenie/data/CustomViolin_params2midi_dev/heuristic_midis/elvis.p'
hpath = '../data/CustomViolin_16k/midi/val/elvis.mid'
groundtruth_path = '/juice/scr/rjcaste/curis/wavegenie/data/CustomViolin_params2midi_dev/midi/elvis.mid'

In [ ]:
gt_midi = pretty_midi.PrettyMIDI(groundtruth_path)
#hmidi = pickle.load(open(hpath, 'rb'))
hmidi = pretty_midi.PrettyMIDI(hpath)

In [ ]:
def midi2_ip(notes):
    """
    Converts notes list (from pretty_midi) to intervals and pitches.
    """
    
    onsets = np.array([n.start for n in notes])
    offsets = np.array([n.end for n in notes])
    pitches = np.array([n.pitch for n in notes])
    intervals = np.vstack((onsets, offsets)).T
    
    return intervals, pitches

Get estimated intervals and pitches...

In [ ]:
#pitches, onsets, offsets = hmidi['pitches'], hmidi['onsets_sec'], hmidi['offsets_sec']

In [ ]:
#est_pitches = np.array(pitches, dtype=int)

In [ ]:
#est_intervals = np.vstack((onsets, offsets)).T

In [ ]:
est_intervals, est_pitches = midi2_ip(hmidi.instruments[0].notes)

Get ground truth intervals and pitches...

In [ ]:
ref_intervals, ref_pitches = midi2_ip(gt_midi.instruments[0].notes)

In [ ]:
mir_eval.transcription.evaluate(ref_intervals, ref_pitches, est_intervals, est_pitches, onset_tolerance=0.10)

In [ ]:
mir_eval.transcription.evaluate(ref_intervals, ref_pitches, est_intervals, np.array([int(round(p)) for p in est_pitches]))

## Debugging

In [ ]:
# synthesize
midi = pretty_midi.PrettyMIDI()

In [ ]:
violin = pretty_midi.Instrument(program=40)

In [ ]:
violin.notes = [pretty_midi.Note(velocity=100, pitch=int(round(p)), start=i[0], end=i[1]) for i, p in zip(est_intervals, est_pitches)]

In [ ]:
violin.notes = violin.notes[:100]

In [ ]:
midi.instruments = [violin]

In [ ]:
out = midi.fluidsynth(fs=16000)
out_gt = gt_midi.fluidsynth(fs=16000)

In [ ]:
from wavegenie.util import preview_audio, DDSP_DEFAULT_FS_AUDIO
start_time = 20
end_time = 25
start_idx = 16000*start_time
end_idx = 16000*end_time
audio = np.stack((out[start_idx:end_idx], out_gt[start_idx:end_idx])).T
preview_audio(audio)

In [ ]:
preview_audio(np.stack((out[16000*start_time:16000*end_time], out[16000*start_time + 100:16000*end_time + 100])).T)

In [ ]:
midi.instruments[0].program = 0

In [ ]:
est_audio = midi.fluidsynth(fs=16000)

In [ ]:
wavwrite('heuristic.wav', 16000, audio)

In [ ]:
ref_audio = gt_midi.fluidsynth(fs=16000)

In [ ]:
ref_audio.shape

In [ ]:
est_audio.shape

In [ ]:
both = np.stack((est_audio[:3034471], ref_audio)).T.astype('float32')

In [ ]:
wavwrite('heuristic.wav', 16000, both)